# SQL query from table names - Continued

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [2]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [7]:
context = [ {'role':'system', 'content':"""
 employees, role, salary, benefits.
"""} ]



In [11]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.

-- Example 1:
-- Question: List all employees and their salaries.
SELECT employees.name, salary.salary
FROM employees
JOIN salary ON employees.ID_usr = salary.ID_usr;

-- Example 2:
-- Question: Show the names of employees who have a salary greater than 50000.
SELECT employees.name
FROM employees
JOIN salary ON employees.ID_usr = salary.ID_usr
WHERE salary.salary > 50000;

-- Example 3:
-- Question: Find the average salary for each educational level.
SELECT studies.educational_level, AVG(salary.salary) as avg_salary
FROM studies
JOIN salary ON studies.ID_usr = salary.ID_usr
GROUP BY studies.educational_level;
"""
})

In [13]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [ ]:
#new
context_user = context.copy()
print(return_CCRMSQL(""" Show the names of employees who have a salary greater than 50000 """, context_user))

Certainly! Below is the SQL query that combines both questions:

```sql
SELECT employees.name, salary.salary, studies.educational_level
FROM employees
JOIN salary ON employees.ID_usr = salary.ID_usr
JOIN studies ON employees.ID_usr = studies.ID_usr
WHERE salary.salary > 50000
GROUP BY employees.name, salary.salary, studies.educational_level;
```

This query will show the names of employees who have a salary greater than 50000 and also display the average salary for each educational level.


In [16]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL(" Find the average salary for each educational level", old_context_user))

This is your SQL:
```sql
SELECT s.ID_usr, AVG(s.salary) AS average_salary, e.name, st.educational_level
FROM salary s
JOIN employees e ON s.ID_usr = e.ID_usr
JOIN studies st ON s.ID_usr = st.ID_usr
GROUP BY st.educational_level;
```

This SQL query retrieves the average salary for each educational level by joining the `salary`, `employees`, and `studies` tables on the corresponding user IDs. It calculates the average salary for each educational level group and displays the educational level along with the average salary.


In [18]:
#new
print(return_CCRMSQL( "Find the average salary for each educational level", context_user))

```sql
SELECT studies.educational_level, AVG(salary.salary) as avg_salary
FROM studies
JOIN salary ON studies.ID_usr = salary.ID_usr
GROUP BY studies.educational_level;
```


In [20]:
#old
print(return_CCRMSQL("  List the project name and the name of the department it belongs to.", old_context_user))

This is your SQL:
```sql
SELECT project_name, department_name
FROM projects
```

This SQL query selects the project name and the name of the department it belongs to from the "projects" table.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

Instruction Clarity Matters, as before.
The Output sometimes is hallucinated when there are no schema.
Prompt variations that don't provide the full CREATE TABLE schema and rely solely on natural language descriptions of tables are much more prone to hallucinating column names or incorrect data types. This was avoided in all three versions by providing the full DDL.
When an LLM does hallucinate or get it wrong, it's often in complex joins or aggregations where the natural language query is ambiguous, or the LLM misinterprets the necessary table relationships.

The exercise confirmed that a well-structured prompt for Text-to-SQL requires a combination of:

Complete Schema Definition: CREATE TABLE statements with data types are fundamental.
Representative Few-Shot Examples: Crucial for demonstrating the desired mapping from natural language to SQL patterns.
Clear Instructions and Constraints: Especially regarding the output format.
Contextual Hints: Explicitly describing table relationships can significantly boost performance for complex queries requiring joins.